<a href="https://colab.research.google.com/github/sayakpaul/TF-2.0-Hacks/blob/master/TF_2_0_and_cloud_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to show how easy it is to serve a machine learning model via [Cloud Functions](https://console.cloud.google.com/functions/) on the Google Cloud Platform. It is absolutely possible to do this via Colab. In this notebook, we will be
- building a simple neural network model to classify the apparels as listed in the FashionMNIST dataset
- serializing the model weights in a way that is compatible with the Cloud Functions' ecosystem
- using the `gcloud` CLI to deploy our model on GCP via Cloud Functions

So, let's get started. 

In [0]:
# install `tensorflow` 2.0 latest
pip install tensorflow==2.0.0b1

In [2]:
# import tensorflow and verify the version
import tensorflow as tf

print(tf.__version__)

2.0.0-beta1


In [0]:
# all the imports we care about in this notebook
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

In [0]:
# load and prepare our data
fashion_mnist = mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
# the humble model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# kickstart the training
model.fit(x_train, y_train, validation_data=(x_test, y_test), 
              epochs=5, batch_size=128,
              verbose=1)


W0711 15:16:04.927937 140034585638784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 42us/sample - loss: 0.5935 - accuracy: 0.7950 - val_loss: 0.4510 - val_accuracy: 0.8406
Epoch 2/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.4178 - accuracy: 0.8515 - val_loss: 0.4079 - val_accuracy: 0.8559
Epoch 3/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3810 - accuracy: 0.8625 - val_loss: 0.3864 - val_accuracy: 0.8592
Epoch 4/5
60000/60000 [==============================] - 2s 36us/sample - loss: 0.3578 - accuracy: 0.8703 - val_loss: 0.3728 - val_accuracy: 0.8656
Epoch 5/5
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3388 - accuracy: 0.8773 - val_loss: 0.3660 - val_accuracy: 0.8660


In [0]:
# save the weights
model.save_weights('fashion_mnist_weights')

This will give birth to two files:
- fashion_mnist_weights.data-00000-of-00001
- fashion_mnist_weights.index

In [17]:
# sample prediction
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
test_img = plt.imread('test.png')
prob = model.predict(test_img.reshape(1, 28, 28))
print(class_names[prob.argmax()])

Trouser


The test image looks like the following, by the way:

![](https://storage.googleapis.com/gweb-cloudblog-publish/images/Cloud_Storage_test_image.max-100x100.png)

Once the model weights are saved we need to create a `.py` file named `main.py` as required by Cloud Functions. The `main.py` file should look like so:

```python
import numpy
import tensorflow as tf

from google.cloud import storage
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from PIL import Image


# we keep model as global variable so we don't have to reload it 
# in case of warm invocations
model = None

def get_me_the_model():
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation='softmax')
])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """downloads a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))

def handler(request):
    global model
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    # model load which only happens during cold invocations
    if model is None:
        download_blob('<your_gs_buckets_name>', 'tensorflow/fashion_mnist_weights.index', '/tmp/fashion_mnist_weights.index')
        download_blob('<your_gs_buckets_name>', 'tensorflow/fashion_mnist_weights.data-00000-of-00001', '/tmp/fashion_mnist_weights.data-00000-of-00001')
        model = get_me_the_model()
        model.load_weights('/tmp/fashion_mnist_weights')
    
    download_blob('<your_gs_buckets_name>', 'tensorflow/test.png', '/tmp/test.png')
    image = numpy.array(Image.open('/tmp/test.png'))
    input_np = (numpy.array(Image.open('/tmp/test.png'))/255)
    input_np = input_np.reshape(1, 28, 28)
    predictions = model.predict(input_np)
    print(predictions)
    print("Image is "+class_names[numpy.argmax(predictions)])
    
    return class_names[numpy.argmax(predictions)]
  ```

**Note** that in place of `<your_gs_buckets_name>` enter the bucket's name (without `gs://`) in which you have stored the model weights. Also note that, I have stored them in a folder named **tensorflow**. When the model is deployed as a cloud function, `main.py`will download the model from the storage bucket and will store it into **tmp** folder. 

Now to get started with the deployment process, first authenticate yourself. 

In [0]:
!gcloud auth login

Set the GCP project (preferably billing enabled).

In [0]:
!gcloud config set project fast-ai-exploration

And deploy!

In [23]:
!gcloud functions deploy handler --runtime python37 --trigger-http --memory 2048
!gcloud functions call handler

availableMemoryMb: 2048
entryPoint: handler
httpsTrigger:
  url: https://us-central1-fast-ai-exploration.cloudfunctions.net/handler
labels:
  deployment-tool: cli-gcloud
name: projects/fast-ai-exploration/locations/us-central1/functions/handler
runtime: python37
serviceAccountEmail: fast-ai-exploration@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-9df9b855-cdbd-46fb-8ddd-787a7047f525/e641be16-f3ca-4d53-b188-8108a000fa07.zip?GoogleAccessId=service-29880397572@gcf-admin-robot.iam.gserviceaccount.com&Expires=1562864691&Signature=n%2F%2F4sB55%2FYRNY%2FpmXQ7PH1wrABZKwpz6XcOlDSBcsauPdciHebBywXv%2FKTvvV9O04tUT%2BrvjXvdfyweMSMcKtHyPYfyaBWIxxlPW89NHEZtzvgQ1l5dGHQv7BNdOqMkbnykFoNAyCbxr8oK5COJaOAXTRbrWR%2FRfVuo9FA6aJnNNqvQAwGbzmrlesyGqcZ0bYNMOOZf2MySjJ1cypDr8UNwuxls2HntAo0yLjyCpDvTynfHKyVlMwn1SegvxOixYZ14lpH%2B8rHfnjWCY9eyZ2cfjcVsx%2FBHZQNgXVT5tST0zA28V0b13Fh1Zkr%2B9EmwCzzVrLeugqmC%2BCZWuLIK%2B%2Fg%3D%3D
status: ACTIVE
timeout: 60s
updateTime: 

**Notice** that the function `handler()` in `main.py` internally calls the test image, so you don't need to worry about it. 

**Reference**:
- https://cloud.google.com/blog/products/ai-machine-learning/how-to-serve-deep-learning-models-using-tensorflow-2-0-with-cloud-functions